
# Predicción de Abandono (Churn) - Proyecto Gym Master

Este notebook predice la probabilidad de abandono (churn) de socios, considerando asistencia, pagos y segmentación de perfil de pago.


In [ ]:

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from supabase import create_client
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Conexión a Supabase
SUPABASE_URL = "https://<TU-PROYECTO>.supabase.co"
SUPABASE_KEY = "<TU-API-KEY>"
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)


In [ ]:

# Cargar datos reales
asistencias = pd.DataFrame(supabase.table('asistencia').select('*').execute().data)
asistencias['fecha'] = pd.to_datetime(asistencias['fecha'])

pagos = pd.DataFrame(supabase.table('pago').select('*').execute().data)
pagos['fecha_pago'] = pd.to_datetime(pagos['fecha_pago'])

segmentos = pd.read_csv('../../Data_Lake_CSV/segmentacion_socios.csv')


In [ ]:

# Etiqueta churn: no asistió en últimos 60 días
hoy = datetime.now()
dias_churn = 60

ultima_asistencia = asistencias.groupby('socio_id')['fecha'].max().reset_index()
ultima_asistencia['dias_ultimo'] = (hoy - ultima_asistencia['fecha']).dt.days
ultima_asistencia['churn'] = ultima_asistencia['dias_ultimo'].apply(lambda x: 1 if x > dias_churn else 0)
ultima_asistencia['churn'].value_counts()


In [ ]:

# Features: total asistencias, promedio días retraso
total_asistencias = asistencias.groupby('socio_id').size().reset_index(name='total_asistencias')

pagos['fecha_vencimiento'] = pd.to_datetime(pagos['fecha_vencimiento'])
pagos['dias_retraso'] = (pagos['fecha_pago'] - pagos['fecha_vencimiento']).dt.days
pagos['dias_retraso'] = pagos['dias_retraso'].apply(lambda x: max(x, 0))

promedio_retraso = pagos.groupby('socio_id')['dias_retraso'].mean().reset_index()
promedio_retraso.rename(columns={'dias_retraso': 'promedio_dias_retraso'}, inplace=True)

df_churn = ultima_asistencia[['socio_id', 'churn']]
df_churn = df_churn.merge(total_asistencias, on='socio_id', how='left')
df_churn = df_churn.merge(promedio_retraso, on='socio_id', how='left')
df_churn = df_churn.merge(segmentos[['socio_id', 'segmento_pago']], on='socio_id', how='left')

segmento_dummies = pd.get_dummies(df_churn['segmento_pago'])
df_churn = pd.concat([df_churn, segmento_dummies], axis=1)
df_churn.fillna(0, inplace=True)

df_churn['churn'].value_counts()


In [ ]:

features = ['total_asistencias', 'promedio_dias_retraso'] + list(segmento_dummies.columns)
X = df_churn[features]
y = df_churn['churn']

# Stratify para garantizar representación de ambas clases
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)
print(classification_report(y_test, y_pred))
print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')


In [ ]:

# Asignar probabilidad de churn solo si el modelo entrenó con ambas clases
if len(modelo.classes_) > 1:
    df_churn['prob_churn'] = modelo.predict_proba(X)[:,1]
else:
    df_churn['prob_churn'] = 0  # o 1 si solo hubo clase churn=1

df_churn[['socio_id', 'prob_churn']].sort_values(by='prob_churn', ascending=False).head()


In [ ]:

# Guardar resultados
df_churn[['socio_id', 'prob_churn']].to_csv('../../Data_Lake_CSV/probabilidad_churn.csv', index=False)
print('✅ Probabilidad de churn guardada en ../../Data_Lake_CSV/probabilidad_churn.csv')



## Conclusión

El modelo ahora maneja correctamente la predicción incluso en casos donde hay desbalance de clases, evitando errores en el cálculo de probabilidades.
